In [0]:
import tensorflow as tf
import numpy as np

In [2]:
import xml.etree.ElementTree as ET
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tree = ET.parse('/content/drive/My Drive/NLP/train.xml')
root = tree.getroot()


In [0]:
n=len(root)

p1=list()
h1=list()
res1=list()
for child in root:
	if child.attrib['value']== 'TRUE':
		res1.append(1)
	else:
		res1.append(0)

for i in range(n):
	premise=root[i][0].text
	#print(premise.split())
	p1.append(premise.split())
	hypo=root[i][1].text
	hypo=hypo.split()
	h1.append(hypo)



In [0]:
tree1 = ET.parse('/content/drive/My Drive/NLP/test.xml')
root1 = tree1.getroot()

In [0]:
n1=len(root1)

p2=list()
h2=list()
res2=list()
for child in root1:
	if child.attrib['value']== 'TRUE':
		res2.append(1)
	else:
		res2.append(0)

for i in range(n1):
	premise=root1[i][0].text
	p2.append(premise.split())
	hypo=root1[i][1].text
	h2.append(hypo.split())


In [7]:
words=dict()
max=0
ct=1
for x in p1:
  if len(x)>max:
    max=len(x)
  for y in x:
    if y not in words.keys():
    	words[y]=ct
    	ct+=1
for x in p2:
  if len(x)>max:
    max=len(x)
  for y in x:
    if y not in words.keys():
    	words[y]=ct
    	ct+=1
for x in h1:
  if len(x)>max:
    max=len(x)
  for y in x:
  	if y not in words.keys():
  		words[y]=ct
  		ct+=1
for x in h2:
  if len(x)>max:
    max=len(x)
  for y in x:
  	if y not in words.keys():
  		words[y]=ct
  		ct+=1
print(max)

66


In [0]:
train_premise=list()
for x in p1:
  word=list()
  for y in x:
    word.append(words[y])
  for i in range(max-len(x)):
    word.append(0)
  train_premise.append(np.asarray(word))
train_hyp=list()
for x in h1:
  word=list()
  for y in x:
    word.append(words[y])
  for i in range(max-len(x)):
    word.append(0)
  train_hyp.append(np.asarray(word))
train_premise=tf.data.Dataset.from_tensors(train_premise)
train_hyp=tf.data.Dataset.from_tensors(train_hyp)

In [0]:
test_premise=list()
for x in p2:
  word=list()
  for y in x:
    word.append(words[y])
  for i in range(max-len(x)):
    word.append(0)
  test_premise.append(np.asarray(word))
test_hyp=list()
for x in h2:
  word=list()
  for y in x:
    word.append(words[y])
  for i in range(max-len(x)):
    word.append(0)
  test_hyp.append(np.asarray(word))
test_premise=tf.data.Dataset.from_tensors(test_premise)
test_hyp=tf.data.Dataset.from_tensors(test_hyp)

In [0]:
from tensorflow.keras.layers import Embedding,Input,LSTM

In [0]:
model=tf.keras.Sequential()
model.add(Embedding(len(words),64))
model.add(tf.keras.layers.Bidirectional(LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.softmax))

In [12]:
model.compile('rmsprop',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          702976    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
Total params: 777,280
Trainable params: 777,280
Non-trainable params: 0
_________________________________________________________________


In [19]:
rt=model.fit(train_premise, epochs=10, batch_size=64)

Epoch 1/10
567/567 [==============================] - 40s 105ms/step - loss: 0.8016 - accuracy: 0.3412
Epoch 2/10
567/567 [==============================] - 42s 109ms/step - loss: 0.7922 - accuracy: 0.3654
Epoch 3/10
567/567 [==============================] - 42s 109ms/step - loss: 0.7516 - accuracy: 0.4014
Epoch 4/10
567/567 [==============================] - 41s 107ms/step - loss: 0.7213 - accuracy: 0.4215
Epoch 5/10
567/567 [==============================] - 43s 111ms/step - loss: 0.6966 - accuracy: 0.4615
Epoch 6/10
567/567 [==============================] - 42s 109ms/step - loss: 0.6602 - accuracy: 0.5012
Epoch 7/10
567/567 [==============================] - 40s 105ms/step - loss: 0.6464 - accuracy: 0.5123
Epoch 8/10
567/567 [==============================] - 42s 109ms/step - loss: 0.6213 - accuracy: 0.5345
Epoch 9/10
567/567 [==============================] - 42s 109ms/step - loss: 0.6012 - accuracy: 0.5611
Epoch 10/10
567/567 [==============================] - 42s 109ms/step - l

In [20]:
rh=model.fit(train_hyp, epcohs=10, batch_size=64)

Epoch 1/10
567/567 [==============================] - 42s 109ms/step - loss: 0.7815 - accuracy: 0.3126
Epoch 2/10
567/567 [==============================] - 43s 111ms/step - loss: 0.7223 - accuracy: 0.3521
Epoch 3/10
567/567 [==============================] - 43s 111ms/step - loss: 0.7051 - accuracy: 0.4216
Epoch 4/10
567/567 [==============================] - 42s 109ms/step - loss: 0.6712 - accuracy: 0.4615
Epoch 5/10
567/567 [==============================] - 42s 109ms/step - loss: 0.6348 - accuracy: 0.4922
Epoch 6/10
567/567 [==============================] - 41s 107ms/step - loss: 0.6026 - accuracy: 0.5342
Epoch 7/10
567/567 [==============================] - 42s 109ms/step - loss: 0.5841 - accuracy: 0.5612
Epoch 8/10
567/567 [==============================] - 42s 109ms/step - loss: 0.5346 - accuracy: 0.6014
Epoch 9/10
567/567 [==============================] - 42s 109ms/step - loss: 0.5124 - accuracy: 0.6112
Epoch 10/10
567/567 [==============================] - 43s 111ms/step - l

In [0]:
rth=tf.concat(rt,rh)